<a href="https://colab.research.google.com/github/hallam-flda/football_data_analysis/blob/main/Scraping_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# FBref scraping
# packages

import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
import time
import re
import random

headers = {'User-Agent': 'Mozilla/5.0'}

# Squad Lists

In [ ]:
# Get for one URL

squad_url = "https://fbref.com/en/squads/822bd0ba/Liverpool-Stats"
squad_tables = pd.read_html(squad_url)


In [ ]:
league_squad_df = squad_tables[0]

prefix_map = {
    "Playing Time": "ptime_",
    "Performance": "perf_",
    "Expected": "exp_",
    "Progression": "prog_",
    "Per 90 Minutes": "p90_"
}


league_squad_df.columns = [
    f"{prefix_map.get(parent.strip(), '')}{child.strip()}" if "Unnamed" not in str(parent) else child.strip()
    for parent, child in league_squad_df.columns
]


In [ ]:
# extracting team name from url
match = re.search(r'/([^/]+)-Stats$', squad_url)
team_name = match.group(1).replace('-', ' ') if match else None
season = squad_tables[1]["Date"][0][:4]

# useful for joining later
league_squad_df['Team'] = team_name
league_squad_df['Season'] = season

# last two rows are totals
league_squad_df = league_squad_df[:-2]

league_squad_df.head()

,Player,Nation,Pos,Age,ptime_MP,ptime_Starts,ptime_Min,ptime_90s,perf_Gls,perf_Ast,...,p90_G-PK,p90_G+A-PK,p90_xG,p90_xAG,p90_xG+xAG,p90_npxG,p90_npxG+xAG,Matches,Team,Season
0,Virgil van Dijk,nl NED,DF,33-277,31,31,2790.0,31.0,1.0,1.0,...,0.03,0.06,0.05,0.02,0.07,0.05,0.07,Matches,Liverpool,2024
1,Mohamed Salah,eg EGY,FW,32-300,31,31,2747.0,30.5,27.0,17.0,...,0.59,1.15,0.74,0.37,1.11,0.51,0.88,Matches,Liverpool,2024
2,Ryan Gravenberch,nl NED,MF,22-330,31,31,2651.0,29.5,0.0,3.0,...,0.00,0.10,0.03,0.10,0.13,0.03,0.13,Matches,Liverpool,2024
3,Alexis Mac Allister,ar ARG,MF,26-108,30,27,2291.0,25.5,4.0,3.0,...,0.16,0.27,0.09,0.16,0.26,0.09,0.26,Matches,Liverpool,2024
4,Andrew Robertson,sct SCO,DF,31-031,29,26,2187.0,24.3,0.0,0.0,...,0.00,0.00,0.04,0.17,0.21,0.04,0.21,Matches,Liverpool,2024


In [ ]:
league_squad_df[:-2]

In [ ]:
test.tail()

,Player,Nation,Pos,Age,ptime_MP,ptime_Starts,ptime_Min,ptime_90s,perf_Gls,perf_Ast,...,p90_G-PK,p90_G+A-PK,p90_xG,p90_xAG,p90_xG+xAG,p90_npxG,p90_npxG+xAG,Matches,Team,Season
24,Harvey Davies,eng ENG,GK,21-220,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,Liverpool,2024
25,James McConnell,eng ENG,MF,20-210,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,Liverpool,2024
26,Tyler Morton,eng ENG,MF,22-162,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,Liverpool,2024
27,Amara Nallo,eng ENG,DF,18-144,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,Liverpool,2024
28,Trey Nyoni,eng ENG,MF,17-285,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Matches,Liverpool,2024


# Team Form - Complete


In [ ]:
league_table_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

df = pd.read_html(league_table_url, header = [0,1])[1]

In [ ]:
df

Unnamed: 0_level_0 Unnamed: 1_level_0 Home                             ...  \
                   Rk              Squad   MP   W  D   L  GF  GA  GD Pts  ...   
0                   1          Liverpool   15  12  2   1  32  11  21  38  ...   
1                   2            Arsenal   15  10  4   1  30  12  18  34  ...   
2                   3    Nott'ham Forest   15   9  4   2  24  10  14  31  ...   
3                   4            Chelsea   15   9  4   2  28  15  13  31  ...   
4                   5      Newcastle Utd   14   8  2   4  26  18   8  26  ...   
5                   6    Manchester City   15   9  3   3  32  19  13  30  ...   
6                   7        Aston Villa   16   8  7   1  27  19   8  31  ...   
7                   8             Fulham   16   7  5   4  25  23   2  26  ...   
8                   9           Brighton   15   6  6   3  21  19   2  24  ...   
9                  10        Bournemouth   15   6  3   6  19  14   5  21  ...   
10                 11     Crystal Palace   16   5  5   6  19  23  -4  20  ...   
11                 12          Brentford   16   7  4   5  30  27   3  25  ...   
12                 13     Manchester Utd   16   6  3   7  21  25  -4  21  ...   
13                 14          Tottenham   16   6  3   7  33  27   6  21  ...   
14                 15            Everton   16   4  8   4  22  19   3  20  ...   
15                 16           West Ham   16   5  3   8  20  30 -10  18  ...   
16                 17             Wolves   15   4  2   9  19  27  -8  14  ...   
17                 18       Ipswich Town   16   1  4  11  13  36 -23   7  ...   
18                 19     Leicester City   16   2  3  11  11  33 -22   9  ...   
19                 20        Southampton   15   1  2  12  11  40 -29   5  ...   

   Away                                                  
      L  GF  GA  GD Pts Pts/MP    xG   xGA   xGD xGD/90  
0     1  40  19  21  35   2.19  36.0  14.6  21.4   1.34  
1     2  26  14  12  28   1.75  22.8  17.0   5.8   0.36  
2     6  27  27   0  26   1.63  19.5  23.5  -4.0  -0.25  
3     6  26  22   4  22   1.38  28.0  25.3   2.6   0.16  
4     5  26  21   5  27   1.69  26.6  19.5   7.1   0.44  
5     6  25  21   4  22   1.38  23.9  22.9   1.0   0.06  
6     7  19  27  -8  20   1.33  16.5  26.3  -9.8  -0.65  
7     5  22  19   3  22   1.47  21.8  18.8   3.0   0.20  
8     5  28  28   0  23   1.44  23.0  24.8  -1.8  -0.11  
9     4  32  26   6  24   1.50  29.9  22.9   7.0   0.43  
10    3  20  12   8  23   1.64  19.2  17.8   1.4   0.10  
11    8  21  20   1  17   1.13  19.6  23.5  -3.9  -0.26  
12    6  16  16   0  17   1.13  17.8  21.6  -3.8  -0.26  
13    9  25  18   7  16   1.07  21.7  22.4  -0.7  -0.05  
14    6  11  19  -8  15   1.00  14.8  20.1  -5.2  -0.35  
15    6  15  22  -7  17   1.13  18.7  21.6  -2.9  -0.19  
16    8  24  32  -8  18   1.13  18.3  25.4  -7.0  -0.44  
17    8  18  29 -11  13   0.87  14.1  33.7 -19.6  -1.30  
18   11  14  37 -23   8   0.53  11.8  30.9 -19.1  -1.27  
19   13  12  34 -22   5   0.31  13.0  33.5 -20.4  -1.28  

[20 rows x 28 columns]

In [ ]:
df.columns = [
    f"{parent.strip()}_{child.strip()}" if "Unnamed" not in str(parent) else child.strip()
    for parent, child in df.columns
]

In [ ]:
df.columns

Index(['Rk', 'Squad', 'Home_MP', 'Home_W', 'Home_D', 'Home_L', 'Home_GF',
       'Home_GA', 'Home_GD', 'Home_Pts', 'Home_Pts/MP', 'Home_xG', 'Home_xGA',
       'Home_xGD', 'Home_xGD/90', 'Away_MP', 'Away_W', 'Away_D', 'Away_L',
       'Away_GF', 'Away_GA', 'Away_GD', 'Away_Pts', 'Away_Pts/MP', 'Away_xG',
       'Away_xGA', 'Away_xGD', 'Away_xGD/90'],
      dtype='object')

# Complete Functions - In progress

## Team Form/League Table - Complete

In [ ]:
def get_league_data(url):
  # standard league table
  league_table_df = pd.read_html(url)[0]

  # home and away form
  league_table_ha_df = pd.read_html(league_table_url, header = [0,1])[1]
  league_table_ha_df.columns = [
    f"{parent.strip()}_{child.strip()}" if "Unnamed" not in str(parent) else child.strip()
    for parent, child in league_table_ha_df.columns
  ]

  return league_table_df, league_table_ha_df




In [ ]:
prem_table, prem_table_ha = get_league_data(league_table_url)

In [ ]:
prem_table.to_csv("prem_table.csv")

## Squad Lists - In Progress

In [ ]:
def get_team_data(url):
  time.sleep(10)
  print(f"trying URL {url}")
  squad_tables = pd.read_html(url)
  league_squad_df = squad_tables[0]

  prefix_map = {
      "Playing Time": "ptime_",
      "Performance": "perf_",
      "Expected": "exp_",
      "Progression": "prog_",
      "Per 90 Minutes": "p90_"
  }


  league_squad_df.columns = [
      f"{prefix_map.get(parent.strip(), '')}{child.strip()}" if "Unnamed" not in str(parent) else child.strip()
      for parent, child in league_squad_df.columns
  ]

  # extracting team name from url
  match = re.search(r'/([^/]+)-Stats$', url)
  team_name = match.group(1).replace('-', ' ') if match else None
  season = squad_tables[1]["Date"][0][:4]

  # useful for joining later
  league_squad_df['Team'] = team_name
  league_squad_df['Season'] = season

  # last two rows are totals
  league_squad_df = league_squad_df[:-2]

  print(f'{team_name} gathered')
  return league_squad_df

In [ ]:
liverpool_df = get_team_data(squad_url)

trying URL https://fbref.com/en/squads/822bd0ba/Liverpool-Stats
Liverpool gathered


In [ ]:
liverpool_df

,Player,Nation,Pos,Age,MP,ptime_Starts,ptime_Min,ptime_90s,perf_Gls,perf_Ast,...,p90_G-PK,p90_G+A-PK,p90_xG,p90_xAG,p90_xG+xAG,p90_npxG,p90_npxG+xAG,Matches,Team,Season
0,Virgil van Dijk,nl NED,DF,33-278,31,31,2790.0,31.0,1.0,1.0,...,0.03,0.06,0.05,0.02,0.07,0.05,0.07,Matches,Liverpool,2024
1,Mohamed Salah,eg EGY,FW,32-301,31,31,2747.0,30.5,27.0,17.0,...,0.59,1.15,0.74,0.37,1.11,0.51,0.88,Matches,Liverpool,2024
2,Ryan Gravenberch,nl NED,MF,22-331,31,31,2651.0,29.5,0.0,3.0,...,0.00,0.10,0.03,0.10,0.13,0.03,0.13,Matches,Liverpool,2024
3,Alexis Mac Allister,ar ARG,MF,26-109,30,27,2291.0,25.5,4.0,3.0,...,0.16,0.27,0.09,0.16,0.26,0.09,0.26,Matches,Liverpool,2024
4,Andrew Robertson,sct SCO,DF,31-032,29,26,2187.0,24.3,0.0,0.0,...,0.00,0.00,0.04,0.17,0.21,0.04,0.21,Matches,Liverpool,2024
5,Trent Alexander-Arnold,eng ENG,DF,26-187,28,26,2145.0,23.8,2.0,6.0,...,0.08,0.34,0.07,0.28,0.35,0.07,0.35,Matches,Liverpool,2024
6,Dominik Szoboszlai,hu HUN,MF,24-169,29,24,2097.0,23.3,5.0,3.0,...,0.21,0.34,0.29,0.21,0.51,0.29,0.51,Matches,Liverpool,2024
7,Ibrahima Konaté,fr FRA,DF,25-322,25,24,2050.0,22.8,1.0,2.0,...,0.04,0.13,0.07,0.04,0.11,0.07,0.11,Matches,Liverpool,2024
8,Luis Díaz,co COL,FW,28-089,30,23,1971.0,21.9,10.0,5.0,...,0.46,0.68,0.37,0.19,0.56,0.37,0.56,Matches,Liverpool,2024
9,Alisson,br BRA,GK,32-192,21,21,1878.0,20.9,0.0,0.0,...,0.00,0.00,0.00,0.03,0.03,0.00,0.03,Matches,Liverpool,2024


In [ ]:
squad_urls = [
    "https://fbref.com/en/squads/18bb7c10/Arsenal-Stats",
    "https://fbref.com/en/squads/8602292d/Aston-Villa-Stats",
    "https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats",
    "https://fbref.com/en/squads/cd051869/Brentford-Stats",
    "https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats",
    "https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats",
    "https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats",
    "https://fbref.com/en/squads/d3fd31cc/Everton-Stats",
    "https://fbref.com/en/squads/fd962109/Fulham-Stats",
    "https://fbref.com/en/squads/b74092de/Ipswich-Town-Stats",
    "https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats",
    "https://fbref.com/en/squads/822bd0ba/Liverpool-Stats",
    "https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats",
    "https://fbref.com/en/squads/19538871/Manchester-United-Stats",
    "https://fbref.com/en/squads/33c895d4/Newcastle-United-Stats",
    "https://fbref.com/en/squads/a2d435b3/Nottingham-Forest-Stats",
    "https://fbref.com/en/squads/33c895d4/Southampton-Stats",
    "https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats",
    "https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats",
    "https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats"
]


In [ ]:
all_prem_squads = []

for url in squad_urls:
  print(f'starting {url}')
  df = get_team_data(url)
  all_prem_squads.append(df)
  print('data appended')

all_prem_squads_df = pd.concat(all_prem_squads)


starting https://fbref.com/en/squads/18bb7c10/Arsenal-Stats
trying URL https://fbref.com/en/squads/18bb7c10/Arsenal-Stats
Arsenal gathered
data appended
starting https://fbref.com/en/squads/8602292d/Aston-Villa-Stats
trying URL https://fbref.com/en/squads/8602292d/Aston-Villa-Stats
Aston Villa gathered
data appended
starting https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats
trying URL https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats
Bournemouth gathered
data appended
starting https://fbref.com/en/squads/cd051869/Brentford-Stats
trying URL https://fbref.com/en/squads/cd051869/Brentford-Stats
Brentford gathered
data appended
starting https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats
trying URL https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats
Brighton and Hove Albion gathered
data appended
starting https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
trying URL https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats
Chelsea gathered
data appended
s

HTTPError: HTTP Error 500: Internal Server Error

In [ ]:
final_squads = squad_urls[-2:]

In [ ]:
for url in final_squads:
  print(f'starting {url}')
  df = get_team_data(url)
  all_prem_squads.append(df)
  print('data appended')


all_prem_squads_df = pd.concat(all_prem_squads)

starting https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats
trying URL https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats
West Ham United gathered
data appended
starting https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats
trying URL https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats
Wolverhampton Wanderers gathered
data appended


In [ ]:
all_prem_squads_df.to_csv("all_prem_squads.csv")